In [5]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import json

class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_data = self.case.get('input', [])
            self.weights = self.case.get('weights', [])
            self.expect = self.data.get('expect', {})
            raw_layers = self.model.get('layers', [])

            self.input_size = self.model.get('input_size')
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]

            self.output = self.expect.get('output', [])
            self.max_sse = self.expect.get('max_sse')

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")


In [7]:
class FastFeedNeuralNetwork:
    def __init__(self, layers, weights):
        self.layers = layers
        self.weights = [np.array(w) for w in weights]

    def softmax(self, x):
        expX = np.exp(x - np.max(x, axis=1, keepdims=True))
        return expX / np.sum(expX, axis=1, keepdims=True)

    def relu(self, x):
        return np.maximum(0, x)
    
    def forward_propagation(self, input_data):
        # Need to add [1] in front of the input_data
        input_data_np = np.array(input_data)
        prepended_input_data = np.insert(input_data_np, 0, 1, axis=1)

        activations = prepended_input_data
        for i in range(len(self.layers)):
            activation_function_str = self.layers[i]['activation_function']
            net_input = np.dot(activations, self.weights[i])

            match activation_function_str:
                case "relu":
                    activations = self.relu(net_input)
                case "softmax":
                    activations = self.softmax(net_input)
                case _:
                    activations = self.relu(net_input)

        return activations

In [8]:
def main():
    # json_path = "Tests/" + input("Input json file name (*.json in Tests folder): ")
    json_path = 'Tests/relu.json'
    parser = JsonModelParser(json_path)
    if parser.data is None:
        print("Failed to load or parse the JSON data.")
    
    ffnn = FastFeedNeuralNetwork(parser.layers, parser.weights)
    output = ffnn.forward_propagation(parser.input_data)
    print("Output:", output)
    sse = np.sum((np.array(parser.output) - output) ** 2)
    print("Sum of Squared Errors (SSE):", sse)

if __name__ == '__main__':
    main()

Output: [[0.05 1.1  0.  ]]
Sum of Squared Errors (SSE): 4.8148248609680896e-33
